### Import Dependencies

In [25]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import add_to_shopping_cart, get_shopping_cart, remove_from_cart, get_formatted_items_context, get_formatted_reviews_context

from langgraph.checkpoint.postgres import PostgresSaver
import psycopg2
from psycopg2.extras import RealDictCursor, execute_batch
import numpy as np

### Item Availability Check Tool

In [26]:
def check_warehouse_availability(items: list[dict]) -> dict:

    """Check availability of items across warehouses, including partial fulfillment options.
    
    Args:
        items: A list of items to check. Each item is a dictionary with keys: product_id, quantity.
        
    Returns:
        A dictionary containing:
        - can_fulfill_completely: bool indicating if all items can be fulfilled from at least one warehouse
        - warehouses_full_fulfillment: list of warehouses that can fulfill the entire order
        - warehouses_partial_fulfillment: list of warehouses with partial availability
        - unavailable_items: list of items that cannot be fulfilled from any warehouse
        - details: detailed breakdown per warehouse with availability for each item
    """
    
    conn = psycopg2.connect(
        host="localhost",
        port=5433,
        database="tools_database",
        user="langgraph_user",
        password="langgraph_password"
    )
    
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
            result = {
                "can_fulfill_completely": False,
                "warehouses_full_fulfillment": [],
                "warehouses_partial_fulfillment": [],
                "unavailable_items": [],
                "details": []
            }
            
            # Check each warehouse for availability
            warehouse_query = """
                SELECT DISTINCT warehouse_id, warehouse_name, warehouse_location
                FROM warehouses.inventory
            """
            cursor.execute(warehouse_query)
            warehouses = cursor.fetchall()
            
            for warehouse in warehouses:
                warehouse_can_fulfill_all = True
                has_any_availability = False
                warehouse_details = {
                    "warehouse_id": warehouse['warehouse_id'],
                    "warehouse_name": warehouse['warehouse_name'],
                    "warehouse_location": warehouse['warehouse_location'],
                    "items": [],
                    "can_fulfill_all": False,
                    "has_partial": False
                }
                
                for item in items:
                    product_id = item['product_id']
                    requested_quantity = item['quantity']
                    
                    # Check availability in this warehouse
                    availability_query = """
                        SELECT product_id, total_quantity, reserved_quantity, available_quantity
                        FROM warehouses.inventory
                        WHERE warehouse_id = %s AND product_id = %s
                    """
                    cursor.execute(availability_query, (warehouse['warehouse_id'], product_id))
                    inventory = cursor.fetchone()
                    
                    available_qty = inventory['available_quantity'] if inventory else 0
                    
                    item_detail = {
                        "product_id": product_id,
                        "requested": requested_quantity,
                        "available": available_qty,
                        "can_fulfill_completely": available_qty >= requested_quantity,
                        "can_fulfill_partially": available_qty > 0 and available_qty < requested_quantity
                    }
                    
                    warehouse_details["items"].append(item_detail)
                    
                    # Track if warehouse can fulfill this item completely
                    if available_qty < requested_quantity:
                        warehouse_can_fulfill_all = False
                    
                    # Track if warehouse has any availability for any item
                    if available_qty > 0:
                        has_any_availability = True
                
                # Categorize warehouse
                if warehouse_can_fulfill_all:
                    warehouse_details["can_fulfill_all"] = True
                    result["warehouses_full_fulfillment"].append({
                        "warehouse_id": warehouse['warehouse_id'],
                        "warehouse_name": warehouse['warehouse_name'],
                        "warehouse_location": warehouse['warehouse_location']
                    })
                elif has_any_availability:
                    warehouse_details["has_partial"] = True
                    result["warehouses_partial_fulfillment"].append({
                        "warehouse_id": warehouse['warehouse_id'],
                        "warehouse_name": warehouse['warehouse_name'],
                        "warehouse_location": warehouse['warehouse_location']
                    })
                
                result["details"].append(warehouse_details)
            
            # Check if any items cannot be fulfilled from any warehouse
            for item in items:
                product_id = item['product_id']
                requested_quantity = item['quantity']
                
                # Get total available quantity across all warehouses
                total_available_query = """
                    SELECT product_id, SUM(available_quantity) as total_available
                    FROM warehouses.inventory
                    WHERE product_id = %s
                    GROUP BY product_id
                """
                cursor.execute(total_available_query, (product_id,))
                total_available = cursor.fetchone()
                
                total_available_qty = total_available['total_available'] if total_available else 0
                
                if total_available_qty < requested_quantity:
                    result["unavailable_items"].append({
                        "product_id": product_id,
                        "requested": requested_quantity,
                        "total_available_across_warehouses": total_available_qty,
                        "shortage": requested_quantity - total_available_qty
                    })
            
            result["can_fulfill_completely"] = len(result["warehouses_full_fulfillment"]) > 0 and len(result["unavailable_items"]) == 0
            
            return result
            
    finally:
        conn.close()

In [27]:
shopping_cart = [
    {
        "product_id": "B09TNXY54Y",
        "quantity": 3
    },
    {
        "product_id": "B0BGLRMPQD",
        "quantity": 2
    },
    {
        "product_id": "B0C4DBSWGW",
        "quantity": 5
    }
]

In [28]:
availability = check_warehouse_availability(shopping_cart)

In [29]:
availability

{'can_fulfill_completely': False,
 'warehouses_full_fulfillment': [],
 'warehouses_partial_fulfillment': [],
 'unavailable_items': [{'product_id': 'B09TNXY54Y',
   'requested': 3,
   'total_available_across_warehouses': 0,
   'shortage': 3},
  {'product_id': 'B0BGLRMPQD',
   'requested': 2,
   'total_available_across_warehouses': 0,
   'shortage': 2},
  {'product_id': 'B0C4DBSWGW',
   'requested': 5,
   'total_available_across_warehouses': 0,
   'shortage': 5}],
 'details': [{'warehouse_id': 'DE-BER-01',
   'warehouse_name': 'Berlin Distribution Center',
   'warehouse_location': 'Berlin, Germany',
   'items': [{'product_id': 'B09TNXY54Y',
     'requested': 3,
     'available': 0,
     'can_fulfill_completely': False,
     'can_fulfill_partially': False},
    {'product_id': 'B0BGLRMPQD',
     'requested': 2,
     'available': 0,
     'can_fulfill_completely': False,
     'can_fulfill_partially': False},
    {'product_id': 'B0C4DBSWGW',
     'requested': 5,
     'available': 0,
     'can

In [19]:
shopping_cart2 = [
    {
        "product_id": "B0B2JYLGNX",
        "quantity": 1
    }
]

In [20]:
availability2 = check_warehouse_availability(shopping_cart2)

In [21]:
availability2

{'can_fulfill_completely': True,
 'warehouses_full_fulfillment': [{'warehouse_id': 'DE-BER-01',
   'warehouse_name': 'Berlin Distribution Center',
   'warehouse_location': 'Berlin, Germany'},
  {'warehouse_id': 'FR-LYO-01',
   'warehouse_name': 'Lyon Regional Warehouse',
   'warehouse_location': 'Lyon, France'},
  {'warehouse_id': 'DE-MUN-01',
   'warehouse_name': 'Munich Logistics Hub',
   'warehouse_location': 'Munich, Germany'},
  {'warehouse_id': 'DE-HAM-01',
   'warehouse_name': 'Hamburg North Warehouse',
   'warehouse_location': 'Hamburg, Germany'}],
 'warehouses_partial_fulfillment': [],
 'unavailable_items': [],
 'details': [{'warehouse_id': 'DE-BER-01',
   'warehouse_name': 'Berlin Distribution Center',
   'warehouse_location': 'Berlin, Germany',
   'items': [{'product_id': 'B0B2JYLGNX',
     'requested': 1,
     'available': 11,
     'can_fulfill_completely': True,
     'can_fulfill_partially': False}],
   'can_fulfill_all': True,
   'has_partial': False},
  {'warehouse_id': 

### Item Reservation Tool

In [22]:
def reserve_warehouse_items(reservations: list[dict]) -> dict:
    
    """Reserve items from multiple warehouses in a single transaction.
    
    Args:
        reservations: A list of reservations. Each reservation is a dictionary with keys:
                     - warehouse_id: The warehouse to reserve from
                     - product_id: The product to reserve
                     - quantity: The quantity to reserve
        
    Returns:
        A dictionary containing:
        - success: bool indicating if all reservations were successful
        - reserved_items: list of successfully reserved items
        - failed_items: list of items that could not be reserved
    """
    
    conn = psycopg2.connect(
        host="localhost",
        port=5433,
        database="tools_database",
        user="langgraph_user",
        password="langgraph_password"
    )
    conn.autocommit = False  # Use transaction
    
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
            result = {
                "success": False,
                "reserved_items": [],
                "failed_items": []
            }
            
            for reservation in reservations:
                warehouse_id = reservation['warehouse_id']
                product_id = reservation['product_id']
                quantity = reservation['quantity']
                
                # Check and lock the inventory row
                check_query = """
                    SELECT warehouse_id, product_id, warehouse_name, warehouse_location, 
                           total_quantity, reserved_quantity, available_quantity
                    FROM warehouses.inventory
                    WHERE warehouse_id = %s AND product_id = %s
                    FOR UPDATE
                """
                cursor.execute(check_query, (warehouse_id, product_id))
                inventory = cursor.fetchone()
                
                if inventory and inventory['available_quantity'] >= quantity:
                    # Update inventory to reserve the items
                    update_query = """
                        UPDATE warehouses.inventory
                        SET reserved_quantity = reserved_quantity + %s
                        WHERE warehouse_id = %s AND product_id = %s
                    """
                    cursor.execute(update_query, (quantity, warehouse_id, product_id))
                    
                    result["reserved_items"].append({
                        "product_id": product_id,
                        "quantity": quantity,
                        "warehouse_id": warehouse_id,
                        "warehouse_name": inventory['warehouse_name'],
                        "warehouse_location": inventory['warehouse_location']
                    })
                else:
                    result["failed_items"].append({
                        "product_id": product_id,
                        "warehouse_id": warehouse_id,
                        "requested": quantity,
                        "available": inventory['available_quantity'] if inventory else 0,
                        "reason": "insufficient_stock" if inventory else "not_in_warehouse"
                    })
            
            # Only commit if all items were successfully reserved
            if len(result["failed_items"]) == 0:
                conn.commit()
                result["success"] = True
            else:
                conn.rollback()
                result["success"] = False
            
            return result
            
    except Exception as e:
        conn.rollback()
        raise e
    finally:
        conn.close()

In [10]:
reservations = [
    {
        "product_id": "B09TNXY54Y",
        "quantity": 3,
        "warehouse_id": "DE-MUN-01"
    },
    {
        "product_id": "B0BGLRMPQD",
        "quantity": 2,
        "warehouse_id": "DE-MUN-01"
    },
    {
        "product_id": "B0C4DBSWGW",
        "quantity": 5,
        "warehouse_id": "DE-MUN-01"
    }
]

In [11]:
reserve_warehouse_items(reservations)

{'success': False,
 'reserved_items': [],
 'failed_items': [{'product_id': 'B09TNXY54Y',
   'warehouse_id': 'DE-MUN-01',
   'requested': 3,
   'available': 0,
   'reason': 'not_in_warehouse'},
  {'product_id': 'B0BGLRMPQD',
   'warehouse_id': 'DE-MUN-01',
   'requested': 2,
   'available': 0,
   'reason': 'not_in_warehouse'},
  {'product_id': 'B0C4DBSWGW',
   'warehouse_id': 'DE-MUN-01',
   'requested': 5,
   'available': 0,
   'reason': 'not_in_warehouse'}]}

In [23]:
reservations = [
    {
        "product_id": "B0B2JYLGNX",
        "quantity": 3,
        "warehouse_id": "DE-BER-01"
    }
]

In [24]:
reserve_warehouse_items(reservations)

{'success': True,
 'reserved_items': [{'product_id': 'B0B2JYLGNX',
   'quantity': 3,
   'warehouse_id': 'DE-BER-01',
   'warehouse_name': 'Berlin Distribution Center',
   'warehouse_location': 'Berlin, Germany'}],
 'failed_items': []}